# Trabajo Tesis: Preprocesamiento faenas

### Librerias y bases

In [77]:
import os
import requests

import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import display, HTML

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
from geopy.distance import distance
from scipy.spatial import KDTree

In [78]:
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 
#os.chdir('C:/Users/admin/OneDrive/Documents/TrabajoTesis')

In [79]:
df = pd.read_excel('Cluster/Faenas25.xlsx')

In [80]:
# Limpiar nombres de columnas
df.columns = ['RutEmpresa' if col == 'RutEmpresa' else col.lstrip() for col in df.columns]

# Limpiar espacios dentro de las columnas (excepto RutEmpresa)
for col in df.select_dtypes(include='object').columns:
    if col != 'RutEmpresa':
        df[col] = df[col].str.strip()

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28319 entries, 0 to 28318
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RutEmpresa                  28319 non-null  object 
 1   NombreEmpresa               28319 non-null  object 
 2   RegionFaena                 28319 non-null  object 
 3   ProvinciaFaena              28319 non-null  object 
 4   ComunaFaena                 28319 non-null  object 
 5   NombreFaena                 28318 non-null  object 
 6   CategoriaFaena              28319 non-null  object 
 7   IdFaena                     28319 non-null  int64  
 8   RegionInstalacion           28319 non-null  object 
 9   ProvinciaInstalacion        28319 non-null  object 
 10  ComunaInstalacion           28319 non-null  object 
 11  NombreInstalacion           28319 non-null  object 
 12  IdTipoInstalacion           28319 non-null  int64  
 13  TipoInstalacion             283

In [82]:
# Convert lat/lon to radians
coords = np.radians(df[['Latitud', 'Longitud']].values)

# Build KDTree
tree = KDTree(coords)

# Radius (10 km → radians)
radius_km = 10
radius_rad = radius_km / 6371.0

# Prepare lists
nearby_counts = []
nearby_categories = []
categoriaA_counts = []

# Loop with progress bar
for i, coord in tqdm(enumerate(coords), total=len(coords), desc="🔍 Calculating 10km radius"):
    idxs = tree.query_ball_point(coord, r=radius_rad)
    idxs = [j for j in idxs if j != i]  # exclude self
    
    categorias_cercanas = df.loc[idxs, 'CategoriaFaena'].tolist()
    
    nearby_counts.append(len(idxs))
    nearby_categories.append(categorias_cercanas)
    
    # Count only 'A'
    categoriaA_counts.append(sum(cat == 'CATEGORIA A' for cat in categorias_cercanas))

# Add results to df
df['faenas_10km'] = nearby_counts
df['categorias_10km'] = nearby_categories
df['categoriaA_10km'] = categoriaA_counts

df.head()


🔍 Calculating 10km radius: 100%|██████████| 28319/28319 [00:15<00:00, 1796.01it/s]


,RutEmpresa,NombreEmpresa,RegionFaena,ProvinciaFaena,ComunaFaena,NombreFaena,CategoriaFaena,IdFaena,RegionInstalacion,ProvinciaInstalacion,...,Cota,Huso,Datum,IdEstado,Estado,Latitud,Longitud,faenas_10km,categorias_10km,categoriaA_10km
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),III,HUASCO,VALLENAR,PLANTA DE BENEFICIO DE MINERALES - VALLENAR,CATEGORIA B,153576,ATACAMA,HUASCO,...,538,19,PSAD-56,8,ACTIVA,-28.566725,-70.740064,72,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
1,76125921-0,CUPRUM RESOURCES CHILE SPA,IV,ELQUI,LA HIGUERA,LA VERDE,CATEGORIA D,20012162,COQUIMBO,ELQUI,...,1200,19,PSAD-56,169,INACTIVA,-29.432920,-70.683699,66,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,VI,COLCHAGUA,LOLOL,MINA LA VENUS,CATEGORIA D,20013211,LIBERTADOR GENERAL BERNARDO OHIGGINS,COLCHAGUA,...,134,19,PSAD-56,12,ABANDONADA,-34.677189,-71.498091,3,"[CATEGORIA D, CATEGORIA D, CATEGORIA C]",0
3,94638000-8,CIA. MRA. DEL PACIFICO S.A.,III,COPIAPO,TIERRA AMARILLA,PLANTA MAGNETITA,CATEGORIA A,33981,ATACAMA,COPIAPO,...,766,19,PSAD-56,8,ACTIVA,-27.520799,-70.320604,541,"[CATEGORIA D, CATEGORIA C, CATEGORIA D, CATEGO...",106
4,6891906-1,GARY NELSON MERCADO MALBRAN,III,CHAÑARAL,CHAÑARAL,DIEGO DAKAR 1/10,CATEGORIA D,20013237,ATACAMA,CHAÑARAL,...,531,19,PSAD-56,8,ACTIVA,-26.411292,-70.414104,164,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0


In [83]:
dfcata = df.loc[df['categoriaA_10km'] > 0, ['RutEmpresa', 'faenas_10km', 'categoriaA_10km', 'categorias_10km']]


In [84]:
# Ordenar por cantidad de CATEGORIA A (descendente)
dfcata_sorted = dfcata.sort_values(by='categoriaA_10km', ascending=False).reset_index(drop=True)
dfcata_sorted

,RutEmpresa,faenas_10km,categoriaA_10km,categorias_10km
0,77762940-9,202,200,"[CATEGORIA A, CATEGORIA A, CATEGORIA A, CATEGO..."
1,77762940-9,200,199,"[CATEGORIA A, CATEGORIA A, CATEGORIA A, CATEGO..."
2,77762940-9,200,199,"[CATEGORIA A, CATEGORIA A, CATEGORIA A, CATEGO..."
3,77762940-9,200,199,"[CATEGORIA A, CATEGORIA A, CATEGORIA A, CATEGO..."
4,77762940-9,200,199,"[CATEGORIA A, CATEGORIA A, CATEGORIA A, CATEGO..."
...,...,...,...,...
10366,76465515-K,58,1,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO..."
10367,78801520-8,183,1,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, SIN CA..."
10368,76011272-0,73,1,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO..."
10369,5956464-1,332,1,"[CATEGORIA C, CATEGORIA C, CATEGORIA C, CATEGO..."


### Limpieza

Nos quedamos solo con las faenas de producción de cobre

In [85]:
df = df[df['RecursoPrimarioInstalacion'] == 'COBRE']
df.shape

(21158, 30)

Dropeo de variables de poca relevancia, es decir que nos sea id o de ubicación, o que nos ayude a analizar en corto plazo

In [86]:
df = df.drop(columns=[
    'RegionFaena', 'ProvinciaFaena', 'ComunaFaena', 'NombreFaena', 'IdFaena',
    'ComunaInstalacion','ProvinciaInstalacion', 'NombreInstalacion', 'RecursoMineroInstalacion', 'RecursoPrimarioInstalacion',
    'TipoRecursoInstalacion', 'IdInstalacion', 'Norte', 'Este', 'Datum', 'IdEstado', 'IdTipoInstalacion'
])

In [87]:
df

,RutEmpresa,NombreEmpresa,CategoriaFaena,RegionInstalacion,TipoInstalacion,Cota,Huso,Estado,Latitud,Longitud,faenas_10km,categorias_10km,categoriaA_10km
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),CATEGORIA B,ATACAMA,TALLERES Y MAESTRANZA,538,19,ACTIVA,-28.566725,-70.740064,72,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
1,76125921-0,CUPRUM RESOURCES CHILE SPA,CATEGORIA D,COQUIMBO,EXPLORACION DE SUPERFICIE,1200,19,INACTIVA,-29.432920,-70.683699,66,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,CATEGORIA D,LIBERTADOR GENERAL BERNARDO OHIGGINS,MINA SUBTERRANEA,134,19,ABANDONADA,-34.677189,-71.498091,3,"[CATEGORIA D, CATEGORIA D, CATEGORIA C]",0
4,6891906-1,GARY NELSON MERCADO MALBRAN,CATEGORIA D,ATACAMA,MINA SUBTERRANEA,531,19,ACTIVA,-26.411292,-70.414104,164,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
12,76448510-6,SLM CASUALIDAD PRIMERA DE EL MANZANO,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,917,19,IRREGULAR OPERATIVA,-30.204069,-71.082232,711,"[CATEGORIA A, CATEGORIA D, CATEGORIA D, CATEGO...",71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28311,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,CAMPAMENTO,1277,19,EN REVISIÓN DE PROYECTO,-31.216132,-71.267759,130,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
28312,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,1199,19,EN REVISIÓN DE PROYECTO,-31.220795,-71.265226,135,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
28313,6188182-4,EDILIO GOMEZ OYARZUN,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,458,19,EN REVISIÓN DE PROYECTO,-31.548972,-71.343902,51,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0
28314,78110627-5,C & C MINERIA SpA,CATEGORIA D,ANTOFAGASTA,POLVORIN,3250,19,EN REVISIÓN DE PROYECTO,-23.044930,-68.557016,3,"[CATEGORIA D, CATEGORIA D, CATEGORIA D]",0


In [88]:
df_sample = df #.sample(frac=0.05, random_state=42).copy() 


### Enriquesimiento

In [89]:
# Coordinates of the 15 cities
cities_coords = {
    "Santiago": (-33.4489, -70.6693),
    "Valparaíso": (-33.0472, -71.6127),
    "Concepción": (-36.8201, -73.0444),
    "La Serena": (-29.9037, -71.2489),
    "Coquimbo": (-29.9533, -71.3395),
    "Antofagasta": (-23.6500, -70.4000),
    "Temuco": (-38.7369, -72.5904),
    "Rancagua": (-34.1708, -70.7400),
    "Iquique": (-20.2307, -70.1350),
    "Puerto Montt": (-41.4694, -72.9420),
    "Talca": (-35.4261, -71.6550),
    "Arica": (-18.4783, -70.3126),
    "Chillán": (-36.6066, -72.1034),
    "Los Ángeles": (-37.4600, -72.3500),
    "Copiapó": (-27.3668, -70.3326),
    "Valdivia": (-39.8196, -73.2459)
}

# Function to find nearest city
def nearest_city(lat, lon):
    closest_city = min(cities_coords.keys(), key=lambda city: geodesic((lat, lon), cities_coords[city]).km)
    return closest_city

# Apply to your dataframe
tqdm.pandas()
df_sample["Closest_City"] = df_sample.progress_apply(
    lambda row: nearest_city(row["Latitud"], row["Longitud"]), axis=1
)

print(df_sample.head())

100%|██████████| 21158/21158 [00:54<00:00, 386.36it/s]

    RutEmpresa                            NombreEmpresa CategoriaFaena  \
0   61703000-4      EMPRESA NACIONAL DE MINERIA (ENAMI)    CATEGORIA B   
1   76125921-0               CUPRUM RESOURCES CHILE SPA    CATEGORIA D   
2   76362163-4  MINERA RUPERTO PATRICIO ORTIZ PALOMINOS    CATEGORIA D   
4    6891906-1              GARY NELSON MERCADO MALBRAN    CATEGORIA D   
12  76448510-6     SLM CASUALIDAD PRIMERA DE EL MANZANO    CATEGORIA D   

                       RegionInstalacion            TipoInstalacion  Cota  \
0                                ATACAMA      TALLERES Y MAESTRANZA   538   
1                               COQUIMBO  EXPLORACION DE SUPERFICIE  1200   
2   LIBERTADOR GENERAL BERNARDO OHIGGINS           MINA SUBTERRANEA   134   
4                                ATACAMA           MINA SUBTERRANEA   531   
12                              COQUIMBO           MINA SUBTERRANEA   917   

    Huso               Estado    Latitud   Longitud  faenas_10km  \
0     19               A

In [90]:

html = "<div style='height:300px; overflow:auto;'>" + df_sample['Closest_City'].value_counts().to_frame().to_html() + "</div>"
display(HTML(html))

,count
Closest_City,
Copiapó,6015
Valparaíso,4017
Coquimbo,3416
Antofagasta,2791
La Serena,2701
Santiago,1049
Iquique,465
Rancagua,374
Arica,313


In [91]:
df_sample

,RutEmpresa,NombreEmpresa,CategoriaFaena,RegionInstalacion,TipoInstalacion,Cota,Huso,Estado,Latitud,Longitud,faenas_10km,categorias_10km,categoriaA_10km,Closest_City
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),CATEGORIA B,ATACAMA,TALLERES Y MAESTRANZA,538,19,ACTIVA,-28.566725,-70.740064,72,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,Copiapó
1,76125921-0,CUPRUM RESOURCES CHILE SPA,CATEGORIA D,COQUIMBO,EXPLORACION DE SUPERFICIE,1200,19,INACTIVA,-29.432920,-70.683699,66,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,La Serena
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,CATEGORIA D,LIBERTADOR GENERAL BERNARDO OHIGGINS,MINA SUBTERRANEA,134,19,ABANDONADA,-34.677189,-71.498091,3,"[CATEGORIA D, CATEGORIA D, CATEGORIA C]",0,Talca
4,6891906-1,GARY NELSON MERCADO MALBRAN,CATEGORIA D,ATACAMA,MINA SUBTERRANEA,531,19,ACTIVA,-26.411292,-70.414104,164,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,Copiapó
12,76448510-6,SLM CASUALIDAD PRIMERA DE EL MANZANO,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,917,19,IRREGULAR OPERATIVA,-30.204069,-71.082232,711,"[CATEGORIA A, CATEGORIA D, CATEGORIA D, CATEGO...",71,La Serena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28311,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,CAMPAMENTO,1277,19,EN REVISIÓN DE PROYECTO,-31.216132,-71.267759,130,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,Coquimbo
28312,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,1199,19,EN REVISIÓN DE PROYECTO,-31.220795,-71.265226,135,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,Coquimbo
28313,6188182-4,EDILIO GOMEZ OYARZUN,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,458,19,EN REVISIÓN DE PROYECTO,-31.548972,-71.343902,51,"[CATEGORIA D, CATEGORIA D, CATEGORIA D, CATEGO...",0,Valparaíso
28314,78110627-5,C & C MINERIA SpA,CATEGORIA D,ANTOFAGASTA,POLVORIN,3250,19,EN REVISIÓN DE PROYECTO,-23.044930,-68.557016,3,"[CATEGORIA D, CATEGORIA D, CATEGORIA D]",0,Antofagasta


In [ ]:
# Initialize geolocator
geolocator = Nominatim(user_agent="truck_routing_demo")

# ↓ Lower delay: from 1s to 0.5s (about 2 requests per second)
reverse = RateLimiter(geolocator.geocode, min_delay_seconds=0.5, max_retries=2, error_wait_seconds=2)

def get_city_coords(city_name):
    try:
        location = geolocator.geocode(city_name + ", Chile")
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {city_name}: {e}")
        return None, None
    return None, None

# Create city mapping
cities = df_sample["Closest_City"].dropna().unique()
city_coords = {}

for c in tqdm(cities, desc="Geocoding city centers"):
    lat, lon = get_city_coords(c)
    city_coords[c] = (lat, lon)

# Query OSRM for travel time
def get_travel_time(lat1, lon1, city):
    lat2, lon2 = city_coords.get(city, (None, None))
    if lat2 is None or lon2 is None:
        return None
    url = f"https://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        if data.get("routes"):
            duration_sec = data["routes"][0]["duration"]
            return duration_sec / 60  # minutes
    except Exception:
        return None
    return None

# Apply to DataFrame
tqdm.pandas()
df_sample["TravelTime_Truck_min"] = df_sample.progress_apply(
    lambda row: get_travel_time(row["Latitud"], row["Longitud"], row["Closest_City"]),
    axis=1
)

print(df_sample[["Latitud", "Longitud", "Closest_City", "TravelTime_Truck_min"]].head())



100%|██████████| 1058/1058 [17:50<00:00,  1.01s/it]

         Latitud   Longitud Closest_City  TravelTime_Truck_min
14100 -18.766115 -70.305866        Arica             64.160000
23857 -31.187356 -71.307579     Coquimbo            186.573333
26455 -32.413654 -70.932892   Valparaíso            140.648333
17161 -22.676943 -70.189357  Antofagasta            132.145000
1557  -32.682402 -70.976678   Valparaíso            116.875000


In [ ]:
df = pd.read_csv("dataSamples/df_sample.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2116 entries, 0 to 2115
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RutEmpresa                  2116 non-null   object 
 1   NombreEmpresa               2116 non-null   object 
 2   CategoriaFaena              2116 non-null   object 
 3   ProvinciaInstalacion        2116 non-null   object 
 4   ComunaInstalacion           2116 non-null   object 
 5   IdTipoInstalacion           2116 non-null   int64  
 6   TipoInstalacion             2116 non-null   object 
 7   RecursoPrimarioInstalacion  2116 non-null   object 
 8   Cota                        2116 non-null   int64  
 9   Huso                        2116 non-null   int64  
 10  IdEstado                    2116 non-null   int64  
 11  Estado                      2116 non-null   object 
 12  Latitud                     2116 non-null   float64
 13  Longitud                    2116 

In [ ]:
desalinadoras = pd.read_excel("Bases/DESALINADORAS.xlsx")

In [ ]:
desalinadoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Latitud   43 non-null     object
 1   Longitud  43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [ ]:
estaciones = pd.read_csv("Bases/Estaciones.csv")

In [ ]:
estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        292 non-null    object 
 1   railway     292 non-null    object 
 2   usage       292 non-null    object 
 3   industrial  292 non-null    object 
 4   lat         292 non-null    float64
 5   lon         292 non-null    float64
dtypes: float64(2), object(4)
memory usage: 13.8+ KB


In [ ]:
puertos = pd.read_excel("Bases/PuertosChile.xlsx")

In [ ]:
puertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitud    14 non-null     float64
 1   Longitud   14 non-null     float64
 2   Localidad  14 non-null     object 
dtypes: float64(2), object(1)
memory usage: 468.0+ bytes


In [ ]:
def clean_coords(df, cols):
    for col in cols:
        df[col] = (
            df[col]
            .astype(str)                             # ensure all entries are string
            .str.replace(',', '.', regex=False)      # replace commas with dots
            .str.replace(r'[^0-9\.\-]', '', regex=True)  # remove non-numeric chars
            .astype(float)                           # convert to float
        )
    return df

df = clean_coords(df, ['Latitud', 'Longitud'])
desalinadoras = clean_coords(desalinadoras, ['Latitud', 'Longitud'])
estaciones = clean_coords(estaciones, ['lat', 'lon'])
puertos = clean_coords(puertos, ['Latitud', 'Longitud'])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import requests
from tqdm import tqdm


# Convert to string first, then replace commas, then to float
df['Latitud'] = df['Latitud'].astype(str).str.replace(',', '.', regex=False).astype(float)
df['Longitud'] = df['Longitud'].astype(str).str.replace(',', '.', regex=False).astype(float)


# 1. Utility Functions


def to_radians(df, lat_col, lon_col):
    """Convert coordinates to radians for Haversine distance."""
    return np.radians(df[[lat_col, lon_col]].values)

def closest_point(main_df, ref_df, main_lat, main_lon, ref_lat, ref_lon, ref_name_col, prefix):
    """Find the closest reference point for each row using BallTree (Haversine distance)."""
    main_coords = to_radians(main_df, main_lat, main_lon)
    ref_coords = to_radians(ref_df, ref_lat, ref_lon)
    tree = BallTree(ref_coords, metric='haversine')
    dist, idx = tree.query(main_coords, k=1)
    dist_km = dist[:, 0] * 6371  # Convert radians to km

    # Add columns with info
    main_df[f"{prefix}_cercana"] = ref_df.iloc[idx[:, 0]][ref_name_col].values if ref_name_col else idx[:, 0]
    main_df[f"Lat_{prefix}_cercana"] = ref_df.iloc[idx[:, 0]][ref_lat].values
    main_df[f"Lon_{prefix}_cercana"] = ref_df.iloc[idx[:, 0]][ref_lon].values
    main_df[f"Dist_{prefix}_km"] = dist_km
    return main_df

def osrm_driving_time(lat1, lon1, lat2, lon2):
    """Estimate driving time (minutes) using the OSRM public API."""
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
        r = requests.get(url, timeout=10)
        data = r.json()
        if "routes" in data and len(data["routes"]) > 0:
            return data["routes"][0]["duration"] / 60  # seconds → minutes
    except Exception:
        pass
    return np.nan

def compute_osrm_times(df, prefix):
    """Compute OSRM travel times for each row with tqdm progress bar."""
    times = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"🚚 Calculando tiempos hacia {prefix}"):
        t = osrm_driving_time(row["Latitud"], row["Longitud"],
                              row[f"Lat_{prefix}_cercana"], row[f"Lon_{prefix}_cercana"])
        times.append(t)
    df[f"DriveTime_{prefix}_min"] = times
    return df

# ===============================
# 2. Pipeline
# ===============================

def compute_accessibility_pipeline(df, desalinadoras, estaciones, puertos):
    """Full end-to-end pipeline: Nearest + OSRM travel time."""
    print("✅ Starting nearest-point and travel-time pipeline...")

    # Ensure numeric coordinates
    desalinadoras["Latitud"] = desalinadoras["Latitud"].astype(float)
    desalinadoras["Longitud"] = desalinadoras["Longitud"].astype(float)

    # Step 1: Find nearest locations
    print("\n📍 Finding nearest reference points...")
    df = closest_point(df, desalinadoras, "Latitud", "Longitud", "Latitud", "Longitud", None, prefix="Desalinadora")
    df = closest_point(df, estaciones, "Latitud", "Longitud", "lat", "lon", "name", prefix="Estacion")
    df = closest_point(df, puertos, "Latitud", "Longitud", "Latitud", "Longitud", "Localidad", prefix="Puerto")

    # Step 2: Compute driving times
    print("\n🛣️  Calculating driving times (this may take a while)...")
    df = compute_osrm_times(df, "Desalinadora")
    df = compute_osrm_times(df, "Estacion")
    df = compute_osrm_times(df, "Puerto")

    print("\n🎯 Pipeline complete! Columns added:")
    print([
        "Desalinadora_cercana", "Dist_Desalinadora_km", "DriveTime_Desalinadora_min",
        "Estacion_cercana", "Dist_Estacion_km", "DriveTime_Estacion_min",
        "Puerto_cercano", "Dist_Puerto_km", "DriveTime_Puerto_min"
    ])
    return df

# ===============================
# 3. Run the pipeline
# ===============================

# df, desalinadoras, estaciones, puertos already defined

df_enriched = compute_accessibility_pipeline(df, desalinadoras, estaciones, puertos)

# Optional: save
# df_enriched.to_csv("faenas_with_accessibility.csv", index=False)


✅ Starting nearest-point and travel-time pipeline...

📍 Finding nearest reference points...

🛣️  Calculating driving times (this may take a while)...


🚚 Calculando tiempos hacia Desalinadora:   0%|          | 0/2116 [00:00<?, ?it/s]

🚚 Calculando tiempos hacia Puerto: 100%|██████████| 2116/2116 [35:16<00:00,  1.00s/it]


🎯 Pipeline complete! Columns added:
['Desalinadora_cercana', 'Dist_Desalinadora_km', 'DriveTime_Desalinadora_min', 'Estacion_cercana', 'Dist_Estacion_km', 'DriveTime_Estacion_min', 'Puerto_cercano', 'Dist_Puerto_km', 'DriveTime_Puerto_min']


In [ ]:
#df_enriched.to_csv("faenas_with_accessibility.csv", index=False)

In [ ]:
# Compute count of other points within 10 km for each row in df_enriched
r_rad = 10 / 6371.0  # 10 km in radians
coords_rad = np.radians(df_enriched[['Latitud', 'Longitud']].values)

btree = BallTree(coords_rad, metric='haversine')
counts = btree.query_radius(coords_rad, r=r_rad, count_only=True)

# Exclude the point itself
df_enriched['faenas_10km'] = (counts - 1).astype(int)

In [ ]:
df_enriched